In [1]:
# Basics of Langchain 

config = '.env.dev'
from dotenv import load_dotenv
load_dotenv(config)
import os
from langchain_openai import AzureChatOpenAI
from langchain_community.document_loaders import UnstructuredURLLoader
from langchain_community.vectorstores import Chroma

system_prompt_template = """You are chatting with an AI assistant. The assistant is here to help you with any questions you have.

- Assume every request is about the location: {location}

Context:
{context}

History: 
{chat_history}

User Request: {request}

"""

llm = AzureChatOpenAI(
    temperature=0.9,
    model_name=os.environ["OPENAI_MODEL_NAME"], #'text-davinci-003'
    openai_api_version=os.environ["OPENAI_API_VERSION"],
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    openai_api_key=os.environ["AZURE_OPENAI_API_KEY"],
    deployment_name=os.environ["OPENAI_DEPLOYMENT_NAME"]
)



In [2]:

from langchain_openai import AzureOpenAIEmbeddings

pdfs = [
    {
        "id": "dc_zoning_code",
        "title": "District of Columbia Zoning Code",
        "url": 'https://planning.dc.gov/sites/default/files/dc/sites/op/publication/attachments/01_Intro.pdf'
    },
    {
        "id": "dc_bike_laws",
        "title": "Pocket Guide to DC Bike Laws",
        "url": 'https://ddot.dc.gov/sites/default/files/dc/sites/ddot/publication/attachments/DC-Bike-Law-Pocket-Guide-Oct2012.pdf'
    }
]
documents = []
for pdf in pdfs:
    loader = UnstructuredURLLoader( urls=[pdf['url']] )
    pages = loader.load_and_split()
    for page in pages:
        page.metadata = dict(
                        source=f'<a href="{pdf["url"]}">{pdf["title"]}</a>'
                    )
    documents.extend(pages)

embedding_function = AzureOpenAIEmbeddings(
    azure_endpoint = os.environ["OPENAI_EMBEDDINGS_BASE"],
    openai_api_key = os.environ["OPENAI_EMBEDDINGS_KEY"],
) 

chroma_directory = f'./chroma_db'
vectorstore = Chroma.from_documents(
    pages, 
    embedding_function,
    persist_directory=chroma_directory
)
from langchain.chains import RetrievalQAWithSourcesChain

retriever = vectorstore.as_retriever()
chain = RetrievalQAWithSourcesChain.from_llm(
    llm, 
    retriever=retriever, 
    return_source_documents=True,
)


In [3]:

information_prompt_template = """
You are a civic AI assistant. You will provide useful information from trusted sources.

"""

request = "Can you ride a bike on sidewalks?"
prompt_query = f'{information_prompt_template}\nRequest: {request}'

response = chain.invoke({"question": prompt_query})
response

{'question': '\nYou are a civic AI assistant. You will provide useful information from trusted sources.\n\n\nRequest: Can you ride a bike on sidewalks?',
 'answer': 'In the District of Columbia, it is legal to ride on the sidewalk outside the central business district. Riders should yield the right-of-way to pedestrians. Riding on sidewalks in the central business district is not advisable, so riders should do so at their own risk.\n',
 'sources': '<a href="https://ddot.dc.gov/sites/default/files/dc/sites/ddot/publication/attachments/DC-Bike-Law-Pocket-Guide-Oct2012.pdf">DC Bike Laws</a>',
 'source_documents': [Document(page_content='n Are buildings required to have bike\n\nparking?\n\nYes. Title 11, Chapter 21 states that, “Bicycle parking shall be provided for office, retail and service uses… The number of bicycle parking spaces shall be at least equal to 5% of the number of automobile spaces…” Residential building bicycle parking is not currently required but it’s a good idea and ma

In [9]:
request = "What is the comprehensive plan?"
prompt_query = f'{information_prompt_template}\nRequest: {request}'

response = chain.invoke({"question": prompt_query})

response

{'question': '\nYou are a civic AI assistant. You will provide useful information from trusted sources.\n\n\nRequest: What is the comprehensive plan?',
 'answer': "Sorry, I don't know.",
 'sources': '',
 'source_documents': [Document(page_content='District Department of Transportation’s (DDOT) mission is to enhance the quality of life for District residents and visitors by ensuring that people, goods, and information move efficiently and safely, with minimal adverse impacts on residents and the environment. DDOT plans, designs, constructs, and maintains the District’s streets, alleys, sidewalks, bridges, traffic signals, street lights, and bicycle facilities. For more information see www.ddot.dc.gov.\n\nMetropolitan Police Department One of the ten largest local police agencies in the United States, the MPDC is the primary law enforcement agency for the District of Columbia. Founded in 1861, the MPDC of today is on the forefront of technological crime fighting advances, from highly dev